In [1]:
import ee
import geemap
import json
import os
#import sys
from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
#sys.path.insert(1, 'D:/!!Research/rgee_test/GEE_LPC/python_scripts')
from grnd_truth_func import file_address_func
from grnd_truth_func import ee_list_func
from grnd_truth_func import naip_savi_endvi
from grnd_truth_func import stdrd_func
from grnd_truth_func import gauss_smooth_func
from grnd_truth_func import dog_sharp
from grnd_truth_func import segmentation_func
from grnd_truth_func import obj_class
from grnd_truth_func import pb_class

In [3]:
ee.Authenticate()

Enter verification code:  4/1AeaYSHDhIdH88cz7z1x6f6fchj8lvfNEfEPWKkg2MTVHkjERTfmtbezkfVY



Successfully saved authorization token.


In [2]:
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

In [8]:
#dunbar_park, nrm_rangeland, or hackberry_lake_ohv
grnd_asset = 'hackberry_lake_ohv'
ground_truth_poly = ee.FeatureCollection(f'projects/ee-jdawsey/assets/{grnd_asset}')

In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [9]:
from grnd_truth_func import stdrd_ref_func

max_clust = 15
shp = ground_truth_poly

standardized = stdrd_ref_func(shp)

vis_param_gauss = {'bands' : ['endvi', 'R', 'G'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}
Map.addLayer(standardized, vis_param_gauss, f'{grnd_asset}')

segmented_image = segmentation_func(standardized)
obj_classed = obj_class(shp, segmented_image, max_clust)


task_obj = ee.batch.Export.image.toDrive(
    image = obj_classed,
    description = f'{grnd_asset}_obj_class_maxClust{max_clust}',
    folder = '!imagery',
    region = shp.geometry(),
    scale = 1,
    crs = None,
    maxPixels = 130000000
)
task_obj.start()

task_stdrd = ee.batch.Export.image.toDrive(
    image = standardized,
    description = f'{grnd_asset}',
    folder = '!imagery',
    region = shp.geometry(),
    scale = 1,
    crs = None,
    maxPixels = 130000000
)
task_stdrd.start() #"""